In [1]:
import numpy as np
from scipy.stats import norm


def em_algorithm(data, max_iter=100, tol=1e-6):
    # Initialize parameters
    n = len(data)
    data = np.array(data)

    # Random initialization
    means = np.array([np.mean(data) - 1, np.mean(data) + 1])  # Mu's
    variances = np.array([np.var(data), np.var(data)])  # Sigmas
    mixing_weights = np.array([0.5, 0.5])  # Phis

    # EM algorithm
    for _ in range(max_iter):
        # E-step: Compute w_ij = P(z_i = j | x_i, theta)
        weights = np.zeros((n, 2))
        for k in range(2):
            weights[:, k] = mixing_weights[k] * norm.pdf(
                data, means[k], np.sqrt(variances[k])
            )

        # Normalize responsibilities
        weights = weights / weights.sum(axis=1, keepdims=True)

        # M-step: Update parameters
        old_means = means.copy()
        old_variances = variances.copy()

        for k in range(2):
            # Update mixing weights: phi_j = 1/n * sum(w_ij)
            mixing_weights[k] = np.mean(weights[:, k])

            # Update means: mu_j = sum(w_ij * x_i) / sum(w_ij)
            means[k] = np.sum(weights[:, k] * data) / np.sum(weights[:, k])

            # Update variances: sigma_j^2 = sum(w_ij * (x_i - mu_j)^2) / sum(w_ij)
            variances[k] = np.sum(weights[:, k] * (data - means[k]) ** 2) / np.sum(
                weights[:, k]
            )

        # Check convergence
        if (
            np.abs(means - old_means).max() < tol
            and np.abs(variances - old_variances).max() < tol
        ):
            break

    return means, variances, mixing_weights


# Data points
data = [
    -0.39,
    0.12,
    0.94,
    1.67,
    1.76,
    2.44,
    3.72,
    4.28,
    4.92,
    5.53,
    0.06,
    0.48,
    1.01,
    1.68,
    1.80,
    3.25,
    4.12,
    4.60,
    5.28,
    6.22,
]

# Run EM algorithm
means, variances, mixing_weights = em_algorithm(data)

# Print results
print("EM Algorithm Results:")
print("-------------------")
print(f"Gaussian 1:")
print(f"  Mean: {means[0]:.4f}")
print(f"  Variance: {variances[0]:.4f}")
print(f"  Mixing weight: {mixing_weights[0]:.4f}")
print(f"\nGaussian 2:")
print(f"  Mean: {means[1]:.4f}")
print(f"  Variance: {variances[1]:.4f}")
print(f"  Mixing weight: {mixing_weights[1]:.4f}")


EM Algorithm Results:
-------------------
Gaussian 1:
  Mean: 1.0832
  Variance: 0.8114
  Mixing weight: 0.5546

Gaussian 2:
  Mean: 4.6559
  Variance: 0.8188
  Mixing weight: 0.4454
